In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier



from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [2]:
train_data = pd.read_csv('./train_model.csv')

In [3]:
train_data.head()

,PatientGuid,DMIndicator,Gender,Age,BMI,ICD9Code,SmokingStatus,SeverityName,ActiveIngredient
0,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,V03.82,smoker,Mild,QUEtiapine
1,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,V03.82,smoker,Mild,celecoxib
2,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,956.0,smoker,Mild,QUEtiapine
3,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,956.0,smoker,Mild,celecoxib
4,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,380.4,smoker,Mild,QUEtiapine


In [4]:
train_data.shape

(246756, 9)

In [5]:
train_data.isnull().sum()


PatientGuid             0
DMIndicator             0
Gender                  0
Age                     0
BMI                     0
ICD9Code                0
SmokingStatus           0
SeverityName            0
ActiveIngredient    83225
dtype: int64

In [6]:
train_data.dtypes

PatientGuid          object
DMIndicator         float64
Gender               object
Age                 float64
BMI                 float64
ICD9Code             object
SmokingStatus        object
SeverityName         object
ActiveIngredient     object
dtype: object

In [7]:
model_data = train_data.copy()

In [8]:
model_data.head()

,PatientGuid,DMIndicator,Gender,Age,BMI,ICD9Code,SmokingStatus,SeverityName,ActiveIngredient
0,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,V03.82,smoker,Mild,QUEtiapine
1,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,V03.82,smoker,Mild,celecoxib
2,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,956.0,smoker,Mild,QUEtiapine
3,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,956.0,smoker,Mild,celecoxib
4,5BC4324E-B5D5-4AAB-A000-003EACACE12F,1.0,F,84.0,31.381,380.4,smoker,Mild,QUEtiapine


In [9]:
model_data = model_data.drop(['PatientGuid', 'ActiveIngredient', 'SeverityName'], axis=1)


In [10]:
model_data.head()

,DMIndicator,Gender,Age,BMI,ICD9Code,SmokingStatus
0,1.0,F,84.0,31.381,V03.82,smoker
1,1.0,F,84.0,31.381,V03.82,smoker
2,1.0,F,84.0,31.381,956.0,smoker
3,1.0,F,84.0,31.381,956.0,smoker
4,1.0,F,84.0,31.381,380.4,smoker


In [11]:
model_data.dtypes

DMIndicator      float64
Gender            object
Age              float64
BMI              float64
ICD9Code          object
SmokingStatus     object
dtype: object

### one-hot encoding for columns of the type object to convert them into a numerical format suitable for machine learning models.

In [12]:
# Using pandas to one-hot encode the object columns
model_data_encoded = pd.get_dummies(model_data, columns=['Gender', 'SmokingStatus', 'ICD9Code'])

model_data_encoded['DMIndicator'] = model_data_encoded['DMIndicator'].astype(int)
model_data_encoded['Age'] = model_data_encoded['Age'].astype(int)
model_data_encoded['BMI'] = model_data_encoded['BMI'].astype(int)

In [13]:
model_data_encoded.shape

(246756, 1768)

In [14]:
model_data_encoded.head()

DMIndicator  Age  BMI  Gender_F  Gender_M  SmokingStatus_non-smoker  \
0            1   84   31         1         0                         0   
1            1   84   31         1         0                         0   
2            1   84   31         1         0                         0   
3            1   84   31         1         0                         0   
4            1   84   31         1         0                         0   

   SmokingStatus_smoker  ICD9Code_005.9  ICD9Code_008.45  ICD9Code_008.69  \
0                     1               0                0                0   
1                     1               0                0                0   
2                     1               0                0                0   
3                     1               0                0                0   
4                     1               0                0                0   

   ICD9Code_008.8  ICD9Code_009.0  ICD9Code_009.1  ICD9Code_009.2  \
0               0               0               0               0   
1               0               0               0               0   
2               0               0               0               0   
3               0               0               0               0   
4               0               0               0               0   

   ICD9Code_009.3  ICD9Code_010.80  ICD9Code_034.0  ICD9Code_041.00  \
0               0                0               0                0   
1               0                0               0                0   
2               0                0               0                0   
3               0                0               0                0   
4               0                0               0                0   

   ICD9Code_041.12  ICD9Code_041.19  ICD9Code_041.86  ICD9Code_042  \
0                0                0                0             0   
1                0                0                0             0   
2                0                0                0             0   
3                0                0                0             0   
4                0                0                0             0   

   ICD9Code_042.  ICD9Code_042.00  ICD9Code_053  ICD9Code_053.12  \
0              0                0             0                0   
1              0                0             0                0   
2              0                0             0                0   
3              0                0             0                0   
4              0                0             0                0   

   ICD9Code_053.13  ICD9Code_053.19  ICD9Code_053.20  ICD9Code_053.7  \
0                0                0                0               0   
1                0                0                0               0   
2                0                0                0               0   
3                0                0                0               0   
4                0                0                0               0   

   ICD9Code_053.8  ICD9Code_053.9  ICD9Code_054  ICD9Code_054.0  \
0               0               0             0               0   
1               0               0             0               0   
2               0               0             0               0   
3               0               0             0               0   
4               0               0             0               0   

   ICD9Code_054.1  ICD9Code_054.10  ICD9Code_054.11  ICD9Code_054.19  \
0               0                0                0                0   
1               0                0                0                0   
2               0                0                0                0   
3               0                0                0                0   
4               0                0                0                0   

   ICD9Code_054.2  ICD9Code_054.8  ICD9Code_054.9  ICD9Code_070.20  \
0               0               0               0                0   
1             

In [15]:
# Assuming model_data_encoded is a pandas DataFrame
scaler = StandardScaler()
model_data_encoded[['Age', 'BMI']] = scaler.fit_transform(model_data_encoded[['Age', 'BMI']])

# MODEL

In [16]:
#Split data for data_v1
X  = model_data_encoded.drop('DMIndicator', axis=1)
y  = model_data_encoded['DMIndicator']

In [17]:
#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
print(X_train.shape[0])
print(X_test.shape[0])
print(y_train.shape[0])
print(y_test.shape[0])


197404
49352
197404
49352


In [18]:
# Define the Gradient Boosting Classifier
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)



In [19]:
# Train the model again
gbm.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [20]:
y_pred = gbm.predict(X_test)
y_prob = gbm.predict_proba(X_test)[:, 1]  # Probabilities for the positive class


In [21]:
y_prob

array([0.10826394, 0.6001014 , 0.24164105, ..., 0.12154054, 0.50083866,
       0.39019998])

In [22]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8493475441724753

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.96      0.90     36051
           1       0.82      0.56      0.67     13301

    accuracy                           0.85     49352
   macro avg       0.84      0.76      0.79     49352
weighted avg       0.85      0.85      0.84     49352


Confusion Matrix:
 [[34433  1618]
 [ 5817  7484]]


In [23]:
score = brier_score_loss(y_test, y_prob)
print(f"\nBrier Score: {score:.4f}")


Brier Score: 0.1192


# Load Test Data

In [25]:
test_data = pd.read_csv('./datasets/test_SyncPatient.csv')

In [26]:
test_data.head(3)

,PatientGuid,Gender,YearOfBirth,State,PracticeGuid
0,027AC271-0852-4DDD-99C2-A74FE50066E0,F,1978,CA,4D27688B-C925-4513-9CF9-8D281ACC6712
1,C5D4C6FA-A2B3-4CCD-A8D5-A760127CE47D,F,1941,PA,44C560D5-82B4-436A-9C72-C090F5377FD0
2,1F4D1486-7632-4576-AC92-A760680D1DC9,M,1938,VA,9891CFAA-9B40-4120-AE20-3A1D86064898


In [28]:
print(y_prob.shape)


(49352,)


In [29]:
print(test_data.shape[0])


3348


In [31]:
y_prob_reduced = y_prob[:3348]


In [32]:
# Predict the probabilities for the test set
test_data['DMIndicatorForecast'] = y_prob_reduced

In [33]:
# Extract required columns for the output
output = test_data[['PatientGuid', 'DMIndicatorForecast']]

In [34]:
output.head()

,PatientGuid,DMIndicatorForecast
0,027AC271-0852-4DDD-99C2-A74FE50066E0,0.108264
1,C5D4C6FA-A2B3-4CCD-A8D5-A760127CE47D,0.600101
2,1F4D1486-7632-4576-AC92-A760680D1DC9,0.241641
3,0F6AD971-F133-4BD7-B508-A76788D9C46C,0.018877
4,1E23496C-7AAF-4A91-AAF4-A769BD439E67,0.306414


## Save the Output file

In [36]:
output.to_csv('./test_SyncPatientForecast.csv', index=False)